<a href="https://colab.research.google.com/github/Nancy-Shi/ICE_Manuscript/blob/main/042924(based_on_case_4_pseudo_code).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 3-Layer Model with Informtion, Behavior, Disease

In [ ]:
#!pip install hypernetx

In [7]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [2]:
import hypernetx as hnx

In [8]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import random
import math as math
from math import log
import seaborn as sns
import pandas as pd
import matplotlib.ticker as ticker


## Part 1: Hypergraph Generation
The following steps generate a hyper graph using the XGI/HyperNetX python package,  following power-law degree distribution for predifined number of nodes n, number of hyperedges num_hyper_edges, degree exponent gamma, using a configuration model with data stored in a dictionary.

In [9]:
# Step 1: Generate Degree Sequence
def generate_degree_sequence(n, gamma, kmin):
    # Generate a random set from the power law distribution
    u = np.random.uniform(size=n)
    degrees = np.ceil((1.0 - u) ** (-1.0 / (gamma - 1.0)))

    # Adjust degrees based on the minimum and maximum degree values
    kmax = int(np.sqrt(n))
    #kmax = int(((gamma-1)/(gamma-2) * n )** (1/gamma))  # max degree
    degrees = degrees[(degrees >= kmin) & (degrees <= kmax)].astype(int)

    # Truncate or pad the sequence to match the length specified
    if len(degrees) >= n:
        degrees = degrees[:n]
    else:
        degrees = np.concatenate((degrees, np.full(n - len(degrees), kmin)))

    return degrees.tolist()

# Step 2: Generate Hyper Edge Size Sequence
def generate_hyper_edge_sizes(degrees, num_hyper_edges):
    total_degrees = sum(degrees)
    hyper_edge_sizes = []

    # Calculate the average size for each hyper edge
    avg_size = total_degrees // num_hyper_edges
    remainder = total_degrees % num_hyper_edges

    # Define the range for the random distribution
    min_size = 2  # Lower bound of the range
    max_size = int(np.sqrt(total_degrees))  # Upper bound of the range
    #max_size = len(degrees) - num_hyper_edges  # Upper bound of the range

    # Generate hyper edge sizes
    for _ in range(num_hyper_edges):
        size = random.randint(min_size, max_size)
        hyper_edge_sizes.append(size)

    return hyper_edge_sizes


# Step 3: Create Copies of Nodes
def create_node_copies(degrees):
    node_copies = []
    for i, degree in enumerate(degrees):
        for _ in range(degree):
            node_copies.append(i)
    return node_copies

# Step 4: Create Copies of Hyper Edges
def create_hyper_edge_copies(hyper_edge_sizes):
    hyper_edge_copies = []
    for i, size in enumerate(hyper_edge_sizes):
        for _ in range(size):
            hyper_edge_copies.append(i)
    return hyper_edge_copies

# Step 5: Randomly Pair Copies without Repeated Pairs
def randomly_pair_copies(node_copies, hyper_edge_copies):
    pairs = []
    paired_hyper_edges = {} # Using a dictionary to track paired hyper-edges with nodes

    for node_copy in node_copies:
        available_hyper_edges = [h for h in hyper_edge_copies if (h, node_copy) not in paired_hyper_edges]

        # If no available hyper-edges left, shuffle the paired hyper-edges and reset
        if not available_hyper_edges:
            paired_hyper_edges = {}
            available_hyper_edges = [h for h in hyper_edge_copies if (h, node_copy) not in paired_hyper_edges]

        # Randomly choose a hyper-edge that has not been paired yet with the current node
        chosen_hyper_edge = random.choice(available_hyper_edges)
        pairs.append((node_copy, chosen_hyper_edge))

        # Add to paired_hyper_edges
        paired_hyper_edges[(chosen_hyper_edge, node_copy)] = True
        hyper_edge_copies.remove(chosen_hyper_edge)

    return pairs

# Step 6: Convert Bipartite Graph to A Hypergraph Dictionary
def convert_to_hypergraph(pairs):
    hypergraph = {}
    for pair in pairs:
        node, hyper_edge = pair
        if hyper_edge in hypergraph:
            hypergraph[hyper_edge].append(node)
        else:
            hypergraph[hyper_edge] = [node]
    return hypergraph


In [10]:
def build_hypergraph(n, gamma, kmin, num_hyper_edges):
    # Step 1: Generate Degree Sequence
    degrees = generate_degree_sequence(n, gamma, kmin)
    print("Degree Sequence: ", degrees)

    # Step 2: Generate Hyper Edge Size Sequence
    hyper_edge_sizes = generate_hyper_edge_sizes(degrees, num_hyper_edges)
    print("Hyper Edge Sizes: ", hyper_edge_sizes)

    # Step 3: Create Copies of Nodes
    node_copies = create_node_copies(degrees)

    # Step 4: Create Copies of Hyper Edges
    hyper_edge_copies = create_hyper_edge_copies(hyper_edge_sizes)

    # Step 5: Randomly Pair Copies
    pairs = randomly_pair_copies(node_copies, hyper_edge_copies)

    # Step 6: Convert Bipartite Graph to Hypergraph
    hyperedge_dict = convert_to_hypergraph(pairs)

    # Print the resulting hypergraph
    print("Hypergraph Dictionary: ", hyperedge_dict)

    return degrees, hyperedge_dict

In [11]:
# Test 2
n2 = 500  # Number of nodes
gamma2 = 2.5  # Power-law exponent
kmin2 = 3  # Minimum degree
num_hyper_edges2 = 100  # Desired number of hyper edges

degrees2, hyperedge_dict2 = build_hypergraph(n2, gamma2, kmin2, num_hyper_edges2)
H2 = hnx.Hypergraph(hyperedge_dict2)

Degree Sequence:  [3, 7, 4, 4, 5, 10, 5, 3, 3, 3, 3, 4, 4, 10, 5, 4, 3, 3, 11, 6, 16, 3, 3, 3, 4, 4, 4, 4, 7, 3, 6, 3, 3, 3, 3, 4, 3, 3, 3, 4, 3, 3, 6, 3, 5, 4, 7, 4, 8, 3, 4, 6, 4, 4, 6, 3, 3, 13, 5, 4, 3, 5, 3, 4, 3, 3, 4, 3, 3, 20, 3, 10, 7, 7, 5, 3, 6, 5, 3, 3, 5, 3, 3, 3, 4, 3, 5, 11, 3, 5, 4, 3, 3, 5, 3, 6, 4, 3, 4, 3, 4, 3, 3, 3, 6, 3, 3, 4, 11, 3, 6, 3, 5, 8, 18, 4, 3, 3, 4, 4, 3, 5, 6, 5, 3, 3, 3, 3, 3, 4, 3, 6, 3, 5, 4, 3, 3, 3, 3, 3, 3, 3, 4, 6, 16, 3, 3, 5, 5, 5, 3, 3, 4, 3, 14, 4, 10, 7, 4, 3, 3, 6, 3, 6, 11, 5, 4, 3, 4, 7, 3, 19, 5, 5, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 

/usr/local/lib/python3.10/dist-packages/hypernetx/classes/hypergraph.py:563: FutureWarning: This method or function will be deprecated in a future release. Public references to this method or function will be removed from the Hypergraph API in a future release.
  self._nodes = self.E.restrict_to_levels([1])


## Part 2: Assign Behavior Status
NP represents the state of no protection, while P represents the state of with protection.

In [12]:
def assign_protection(hypergraph, fraction_protected):
    # Get the list of nodes from the hypergraph
    nodes = list(hypergraph.nodes())

    # Calculate the number of nodes to protect
    num_nodes_to_protect = int(len(nodes) * fraction_protected)

    # Randomly choose nodes to protect
    nodes_to_protect = random.sample(nodes, num_nodes_to_protect)

    # Initialize the protection status dictionary
    protection_status = {}

    # Assign protection status to each node
    for node in nodes:
        if node in nodes_to_protect:
            protection_status[node] = "P"  # Protected node
        else:
            protection_status[node] = "N"  # Non-protected node

    #print(protection_status)

    return protection_status

In [13]:
# Test:
fraction_protected = 0.1
protection_status_dict = assign_protection(H2, fraction_protected)
print(protection_status_dict)

{0: 'P', 1: 'N', 2: 'N', 3: 'N', 4: 'N', 5: 'P', 6: 'N', 7: 'N', 8: 'N', 9: 'N', 10: 'N', 11: 'N', 12: 'N', 13: 'N', 14: 'N', 15: 'P', 16: 'N', 17: 'N', 18: 'P', 19: 'N', 20: 'N', 21: 'N', 22: 'N', 23: 'N', 24: 'N', 25: 'N', 26: 'P', 27: 'N', 28: 'N', 29: 'N', 30: 'N', 31: 'P', 32: 'N', 33: 'N', 34: 'N', 35: 'N', 36: 'P', 37: 'N', 38: 'N', 39: 'N', 40: 'N', 41: 'N', 42: 'N', 43: 'N', 44: 'N', 45: 'N', 46: 'N', 47: 'N', 48: 'N', 49: 'N', 50: 'N', 51: 'N', 52: 'N', 53: 'N', 54: 'N', 55: 'N', 56: 'N', 57: 'N', 58: 'N', 59: 'N', 60: 'N', 61: 'N', 62: 'N', 63: 'N', 64: 'N', 65: 'N', 66: 'N', 67: 'N', 68: 'N', 69: 'P', 70: 'N', 71: 'N', 72: 'N', 73: 'P', 74: 'N', 75: 'N', 76: 'N', 77: 'N', 78: 'N', 79: 'N', 80: 'N', 81: 'N', 82: 'N', 83: 'N', 84: 'N', 85: 'P', 86: 'N', 87: 'N', 88: 'N', 89: 'P', 90: 'N', 91: 'N', 92: 'P', 93: 'N', 94: 'N', 95: 'N', 96: 'N', 97: 'N', 98: 'N', 99: 'N', 100: 'N', 101: 'N', 102: 'N', 103: 'N', 104: 'N', 105: 'N', 106: 'N', 107: 'N', 108: 'N', 109: 'N', 110: 'N',


## Part 3: Assign Threshold
The following steps assigns a threshold value to each node in the network. The threshold follows a uniform or normal distribution with predefined mean (mu) and standard deviation (sigma).

In [14]:
# Defines the parameters to be used
mu = 0.1
sigma = 0.05

# Function to assign thresholds to the individual nodes
def assign_thresholds(hypergraph, mu, sigma):
    NV = hypergraph.order()
    Ltre = {}

    for node in hypergraph.nodes():
          # Uniform distribution: #
          #Ltre[node] = np.random.uniform()
          # Normal distrution
          while True:
              threshold = random.gauss(mu, sigma)
              if 0 < threshold < 1:
                  break
          Ltre[node] = threshold

    return Ltre

In [15]:
Ltre2 = assign_thresholds(H2, mu, sigma)

print("Threshold List for Nodes: ", Ltre2 )

Threshold List for Nodes:  {0: 0.11811616454190373, 1: 0.04854581204844911, 2: 0.13796137955287333, 3: 0.0978197221986096, 4: 0.10768837267447245, 5: 0.11679855294045884, 6: 0.07729771962910281, 7: 0.07853098395387095, 8: 0.10451027982056009, 9: 0.11475988654176142, 10: 0.07281860517218322, 11: 0.05113176933415074, 12: 0.12155270646562594, 13: 0.029502871561721836, 14: 0.15563554057122267, 15: 0.0949523197188293, 16: 0.07078752963843965, 17: 0.2008936829902046, 18: 0.16501437666705865, 19: 0.058738705296956836, 20: 0.13519299957430844, 21: 0.0392253926426051, 22: 0.035705200652056276, 23: 0.03841539280065729, 24: 0.1857696992328469, 25: 0.022325522346926127, 26: 0.06391769292569748, 27: 0.07380074003293238, 28: 0.24868691710399354, 29: 0.14869187984136314, 30: 0.1225658606594715, 31: 0.05595935093161277, 32: 0.0481221921658152, 33: 0.1794493633373291, 34: 0.0672937052789227, 35: 0.10370597229299539, 36: 0.03371997823263077, 37: 0.10786497616943068, 38: 0.14359586999401708, 39: 0.108694

# Part 4: The ICE Model (The Information Cognition Epidemics Model)
## Information Layer
The misinformation spread occurs on a hyperedge network involving group spreading. The three stages are U(unaware), G(gossip/spreader), and C(stifler/corrected).  

## Cognition Layer
In the cognitive behavioral layer, P is protected, and N is not protected. The rate of transition from state P to N, p, depends on the information layer. The rate from NP to P is 1-p. The transition rate of a node is also affected by the number of active spreader/stiflers. The bigger number of active neighbors, the faster the rate. Another way behavior may change is based on the fraction of protected neighbors.

## Epidemics Layer
In the epidemics layer, the possible disease states are S(susceptible), I(infected), and R(recovered). The illness spreading is pairwise. The disease propagation rate depends on the fraction of protected individuals $\rho_P$.



In [16]:
n = 500  # Number of nodes

# Information Layer
gamma_i = 2.5  # Power-law exponent
kmin_i = 3  # Minimum degree
num_hyper_edges_i = 450  # Desired number of hyper edges
ldeg_i, hyperedge_dict_i = build_hypergraph(n, gamma_i, kmin_i, num_hyper_edges_i)
inw = hnx.Hypergraph(hyperedge_dict_i)
ltre = assign_thresholds(inw, 0.20, 0.05)
print("Acceptance Threshold Sequence: ", ltre)

Degree Sequence:  [3, 3, 4, 3, 3, 3, 3, 3, 3, 4, 12, 15, 6, 3, 22, 7, 22, 3, 3, 18, 4, 3, 3, 3, 4, 3, 3, 4, 3, 17, 6, 4, 3, 4, 5, 4, 5, 5, 6, 3, 16, 4, 3, 3, 3, 3, 4, 3, 3, 4, 3, 6, 3, 6, 3, 22, 4, 8, 5, 3, 3, 14, 6, 3, 3, 4, 7, 6, 3, 9, 3, 8, 3, 7, 3, 3, 5, 4, 3, 3, 11, 4, 8, 3, 4, 8, 9, 4, 5, 3, 3, 4, 3, 3, 3, 13, 6, 3, 3, 4, 4, 3, 3, 20, 4, 7, 5, 3, 14, 5, 5, 7, 7, 5, 19, 3, 4, 3, 3, 5, 4, 6, 4, 6, 4, 3, 4, 16, 7, 3, 5, 3, 3, 3, 3, 4, 4, 7, 3, 6, 3, 4, 3, 6, 3, 3, 4, 8, 3, 4, 3, 3, 13, 5, 3, 3, 3, 6, 3, 5, 3, 6, 5, 17, 3, 8, 4, 3, 4, 3, 4, 3, 3, 4, 3, 5, 10, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 

/usr/local/lib/python3.10/dist-packages/hypernetx/classes/hypergraph.py:563: FutureWarning: This method or function will be deprecated in a future release. Public references to this method or function will be removed from the Hypergraph API in a future release.
  self._nodes = self.E.restrict_to_levels([1])


In [17]:
# Cognition Layer
gamma_c = 3.0  # Power-law exponent
kmin_c = 3  # Minimum degree
ldeg_c = generate_degree_sequence(n, gamma_c, kmin_c)
print("Behavior Degree Sequence: ", ldeg_c)
cnw = nx.configuration_model(ldeg_c)
frac_prot = 0.05
lprot = assign_protection(cnw, frac_prot)

Behavior Degree Sequence:  [3, 4, 3, 4, 3, 3, 7, 4, 3, 7, 3, 5, 3, 3, 3, 3, 6, 3, 3, 6, 3, 3, 3, 11, 5, 4, 7, 4, 3, 3, 3, 3, 5, 3, 4, 4, 3, 3, 3, 3, 4, 10, 3, 3, 3, 3, 4, 3, 4, 3, 4, 3, 3, 3, 3, 6, 4, 6, 5, 3, 3, 3, 3, 4, 3, 3, 3, 11, 3, 3, 4, 8, 3, 5, 3, 3, 4, 3, 3, 3, 3, 4, 3, 3, 4, 3, 4, 3, 3, 3, 5, 7, 3, 10, 3, 4, 4, 7, 5, 3, 4, 4, 3, 3, 4, 7, 3, 4, 7, 4, 4, 3, 4, 3, 3, 3, 3, 3, 17, 7, 3, 4, 4, 3, 3, 4, 3, 3, 3, 4, 3, 4, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3

In [23]:
# Epidemic Layer
gamma_e = 4.0
kmin_e = 3
ldeg_e = generate_degree_sequence(n, gamma_e, kmin_e)
print("Degree Sequence: ", ldeg_e)
enw = nx.configuration_model(ldeg_e)

Degree Sequence:  [3, 3, 3, 4, 8, 3, 3, 5, 3, 4, 3, 4, 4, 9, 3, 3, 7, 4, 3, 3, 3, 3, 3, 4, 6, 3, 3, 3, 3, 5, 6, 3, 3, 4, 3, 3, 4, 3, 3, 3, 4, 3, 4, 4, 3, 3, 4, 3, 3, 3, 4, 3, 3, 3, 3, 5, 3, 4, 3, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 

In [24]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [28]:
def ICE_model_no_control(inw, ldeg_i, ltre, cnw, ldeg_c, lprot, enw, ldeg_e, lam, alp, zeta_1, zeta_2, zeta_3, zeta_4, beta_PP, beta_NP, beta_PN, beta_NN, mu, n_sample):
  """
  Input:
      inw - information hyperedge network
      ltre - list of thresholds for informaiton spread
      ldeg_i - degree sequence of information layer

      cnw - cognitive network
      lprot - list of protection status
      ldeg_c - degree sequence of cognition layer

      enw - epidemic pairwise network
      ldeg_e - degree sequence of epidemic layer

      lambda - information spreading rate
      alp - informaiton stifling rate

      zeta_1 - removing protection rate based on information
      zeta_3 - removing protection rate based on neighborhood behavior
      zeta_2 - adopting protection rate based on information
      zeta_4 - adopting protection rate based on neighborhood behavior

      beta_PP - disease transmission rate between protected S and protected I
      beta_NP - disease transmission rate between not protected S and protected I
      beta_PN - disease transmission rate between protected S and not protected I
      beta_NN - disease transmission rate between not protected S and not protected I

      mu - disease recovery rate

      n_sample - number of samples
  """

  t_max = 1000      # Set maximum time
  kmax_i = max (ldeg_i)     # Get maximum hyperedge degree in information layer
  kmax_c = max (ldeg_c)     # Get maximum hyperedge degree in cognition layer
  kmax_e = max (ldeg_e)     # Get maximum degree in epidemic layer
  N = inw.order()  # Get the network size

  rho_C = []   # Keep track of fraction of corrected in information layer
  rho_P = []   # Keep track of fraction of protected in cognition layer
  rho_R = []   # Keep track of fraction of recovered in epidemic layer

  for i_samp in range(1, n_sample + 1):
      t = 0                 # Initialize time, number of corrected, number of recovered
      N_corrected = 0
      N_recovered = 0

      info_states = {j: "U" for j in inw.nodes()}   # Initialize information and disease states
      disease_states = {k: "S" for k in enw.nodes()}

      lprot = assign_protection(cnw, frac_prot)
      protected = list(filter(lambda node: lprot[node] == "P", lprot))
      N_protected = len(protected)
      not_protected = list(filter(lambda node: lprot[node] == "N", lprot))


      gossip = []     # Create lists to store gossip and corrected individuals in information layer
      corrected = []

      N_gossip = 0
      N_e_i = 0
      for _ in range(10):
        rumor_node_0 = np.random.choice([node for node, state in info_states.items() if state == "U" and node in inw.nodes()])
        info_states[rumor_node_0] = "G"
        gossip.append(rumor_node_0)
        N_gossip += 1
        N_e_i += inw.degree(rumor_node_0)

      infected = []     # Create lists to store infected and recovered individuals in epidemic layer
      recovered = []
      time_record= [0]
      prevalence_count = [0]
      recovered_count = [0]

      N_infected = 0
      N_e_e = 0
      for _ in range(10):
        ill_node_0 = np.random.choice([node for node, state in disease_states.items() if state == "S" and node in enw.nodes()])
        disease_states[ill_node_0] = "I"
        infected.append(ill_node_0)
        N_infected += 1
        N_e_e += enw.degree(ill_node_0)

      #sum_of_beta = beta_PP + beta_NP + beta_PN + beta_NN
      #Silvio:
      beta_max = max(beta_PP,beta_NP,beta_PN,beta_NN) # Chose the largest infection rate (I known that it is beta_NN, but keep it general)
      #sum_of_beta=beta_max                          # You can chose other name for this variable

      while t < 150 and N_infected > 0:   # We stop when there is no infection
        #total_rate = lam * N_e_i + 2 * alp * N_e_i + sum_of_beta * N_e_e + mu * N_infected + zeta_1 * N_protected + zeta_2 * (N-N_protected) + zeta_3 * N_protected + zeta_4 * (N-N_protected)
        total_rate = lam * N_e_i + 2 * alp * N_e_i + beta_max * N_e_e + mu * N_infected + zeta_1 * N_protected + zeta_2 * (N-N_protected) + zeta_3 * N_protected + zeta_4 * (N-N_protected)
        tau = -np.log(np.random.uniform(1e-6, 1)) / total_rate
        t += tau

        if t >= t_max:
                break

      # Determine which event occurs
        event = np.random.uniform()
        p1 = (lam * N_e_i) / total_rate     # rumor spreading
        p2 = (lam * N_e_i + alp * N_e_i) / total_rate  # rumor stifling (by meeting stifling neighbor threshold)
        p3 = (lam * N_e_i + 2 * alp * N_e_i) / total_rate  # rumor stifling (by meeting gossip neighbor threshold)

        p4 = (lam * N_e_i + 2 * alp * N_e_i + beta_max * N_e_e) / total_rate  # disease propagation
        p5 = (lam * N_e_i + 2 * alp * N_e_i + beta_max * N_e_e + mu * N_infected) / total_rate  # disease recovery

        p6 = (lam * N_e_i + 2 * alp * N_e_i + beta_max * N_e_e + mu * N_infected + zeta_1 * N_protected) / total_rate # change to not adopting protection by information
        p7 = (lam * N_e_i + 2 * alp * N_e_i + beta_max * N_e_e + mu * N_infected + zeta_1 * N_protected + zeta_2 * (N-N_protected) ) / total_rate # change to adopting protection by information
        p8 = (lam * N_e_i + 2 * alp * N_e_i + beta_max * N_e_e + mu * N_infected + zeta_1 * N_protected + zeta_2 * (N-N_protected)  + zeta_3 * N_protected) / total_rate # change to not adopting protection by neighborhood behavior

        #p4 = (lam * N_e_i + 2 * alp * N_e_i + sum_of_beta * N_e_e) / total_rate  # disease propagation
        #p5 = (lam * N_e_i + 2 * alp * N_e_i + sum_of_beta * N_e_e + mu * N_infected) / total_rate  # disease recovery

        #p6 = (lam * N_e_i + 2 * alp * N_e_i + sum_of_beta * N_e_e + mu * N_infected + zeta_1 * N_protected) / total_rate # change to not adopting protection by information
        #p7 = (lam * N_e_i + 2 * alp * N_e_i + sum_of_beta * N_e_e + mu * N_infected + zeta_1 * N_protected + zeta_2 * (N-N_protected) ) / total_rate # change to adopting protection by information
        #p8 = (lam * N_e_i + 2 * alp * N_e_i + sum_of_beta * N_e_e + mu * N_infected + zeta_1 * N_protected + zeta_2 * (N-N_protected)  + zeta_3 * N_protected) / total_rate # change to not adopting protection by neighborhood behavior
          # > p8 # change to adopting protection by neighborhood behavior
          #print(p1, p2, p3, p4, p5, p6, p7, p8)

          # Determine if accept selected individual based on degree distribution
        q_deg_i = np.random.uniform()
        q_deg_c = np.random.uniform()
        q_deg_e = np.random.uniform()

          # Case 1: Rumor spreading
        if event < p1:
              ##### old code #####
              #gossip_node = np.random.choice(gossip)
              #while q_deg_i > inw.degree(gossip_node) / kmax_i:
              #    gossip_node = np.random.choice(gossip)
              #if q_deg_i < inw.degree(gossip_node) / kmax_i:
              ####################
                response='F'
                while len(gossip)>0 and response =='F':
                  gossip_node = np.random.choice(gossip) #chose an infected node proportional to the degree
                  draw_rn = np.random.uniform()
                  if draw_rn < inw.degree(gossip_node)/kmax_i:
                     response='T'

                edges_containing_gossip = [edge for edge in inw.edges() if gossip_node in inw[edge]]
                if edges_containing_gossip:
                        rumor_hyper_edge = np.random.choice(edges_containing_gossip)
                        neighbors = inw[rumor_hyper_edge]
                        for neighbor in neighbors:
                            if info_states[neighbor] == "U":
                                count_gossip_neighbors = sum(1 for node in inw.neighbors(neighbor) if info_states[node] == "G")
                                if count_gossip_neighbors / len(inw.neighbors(neighbor)) >= ltre[neighbor]:
                                    info_states[neighbor] = "G"  # uninformed neighbor becomes gossip spreader
                                    gossip.append(neighbor)
                                    N_gossip += 1
                                    N_e_i += inw.degree(neighbor)

          # Case 2: Rumor stifling (by meeting stifling neighbor threshold)
        elif event < p2:
              ##### old code #####
              #stifler_node = np.random.choice(gossip)
              #while q_deg_i > inw.degree(stifler_node)  / kmax_i:
              #     stifler_node = np.random.choice(gossip)
              #if q_deg_i < inw.degree(stifler_node)  / kmax_i:
              ####################
                    response='F'
                    while len(gossip)>0 and response =='F':
                      stifler_node = np.random.choice(gossip) #chose an infected node proportional to the degree
                      draw_rn = np.random.uniform()
                      if draw_rn < inw.degree(stifler_node)/kmax_i:
                         response='T'

                    stifler_hyper_edge = np.random.choice(list(inw.edges()))
                    neighbors = inw[stifler_hyper_edge]
                    count_stifler_neighbors = sum(1 for node in inw.neighbors(stifler_node) if info_states[node] == "C")
                    if count_stifler_neighbors > 0:
                            info_states[stifler_node] = "C"
                            N_gossip -= 1
                            gossip.remove(stifler_node)
                            corrected.append(stifler_node)
                            N_corrected += 1
                            N_e_i -= inw.degree(stifler_node)

          # Case 3: Rumor stifling (by meeting gossip neighbor threshold)
        elif event < p3:
              ##### old code #####
              #stifler_node = np.random.choice(gossip)
              #while q_deg_i > inw.degree(stifler_node) / kmax_i:
              #     stifler_node = np.random.choice(gossip)
              #if q_deg_i < inw.degree(stifler_node) / kmax_i:
              ####################
                    response='F'
                    while len(gossip)>0 and response =='F':
                      stifler_node = np.random.choice(gossip) #chose an infected node proportional to the degree
                      draw_rn = np.random.uniform()
                      if draw_rn < inw.degree(stifler_node)/kmax_i:
                         response='T'

                    stifler_hyper_edge = np.random.choice(list(inw.edges()))
                    neighbors = inw[stifler_hyper_edge]
                    count_gossip_neighbors = sum(1 for node in inw.neighbors(stifler_node) if info_states[node] == "G")
                    if count_gossip_neighbors > 0:
                            info_states[stifler_node] = "C"
                            N_gossip -= 1
                            gossip.remove(stifler_node)
                            corrected.append(stifler_node)
                            N_corrected += 1
                            N_e_i -= inw.degree(stifler_node)

          # Case 4: Disease propagation
        elif event < p4:
            # code based on the new pseudo code
            response='F'
            while len(infected)>0 and response =='F':  #draw node until degree distribution is reached and while the infected list is not empty
              infected_node = np.random.choice(infected) #choose an infected node proportional to the degree
              draw_rn = np.random.uniform()
              if draw_rn < enw.degree(infected_node)/kmax_e:
                response='T'

            neighbors = list(enw.neighbors(infected_node)) #list of neighbors of the infected node
            target_node = np.random.choice(neighbors) #choose a neighbor at random
            # beta_NN > beta_PN > beta_NP > beta_PP
            if disease_states[target_node] == "S":
                if lprot[target_node] == "P" and lprot[infected_node] == "P":
                   beta_correct = beta_PP
                elif lprot[target_node] == "N" and lprot[infected_node] == "P":
                   beta_correct = beta_NP
                elif lprot[target_node] == "P" and lprot[infected_node] == "N":
                   beta_correct = beta_PN
                else:
                   beta_correct = beta_NN

                draw_rn = np.random.uniform()
                if draw_rn < beta_correct/beta_max:
                   disease_states[target_node] = "I"
                   infected.append(target_node)
                   N_infected += 1
                   N_e_e += enw.degree(target_node)
                   time_record.append(t)
                   prevalence_count.append(N_infected)
                   recovered_count.append(N_recovered)

            ############## new pseudo code by Silvio #################################
            #response='F'
            #while response='F':
            #  infected_node = np.random.choice(infected) #chose an infected node proportional to the degree
            #  draw_rn = np.random.uniform()
            #  if draw_rn< degree(infected_node)/max_degree:
            #    reponse='T'
            #
            #neighbors = list(enw.neighbors(infected_node)) #list of neighbors of the infected node
            #target_node = np.random.choice(neighbors) #chose a neighbor at random
            #if disease_states.get(target)=='S':
            #    if infected_node == 'P' and target_node == 'P': #I don't know how to code this
            #      beta_correct=beta_PP
            #    elif infected_node == 'P' and target_node == 'N':
            #      beta_correct=beta_PN
            #    elif infected_node == 'N' and target_node == 'P':
            #      beta_correct=beta_NP
            #    else:
            #      beta_correct=beta_NN
            #
            #    draw_rn = np.random.uniform()
            #    if draw_rn<beta_correct/beta_max:
            #        infect target_node #I don't know how to do this
            #    else:
            #        do nothing - only increase time wherever you do it
            #else:
            #    do nothing - only increase time wherever you do it
            #
            ###################################################################
            ############## old code ###########################################
            #draw_infection_event = np.random.uniform()
            # Infection event #1: between S_P and I_P
            #if draw_infection_event < beta_PP/sum_of_beta:
            #    infected_protected = list(set(infected).intersection(protected))
            #    if len(infected_protected)>0:
            #      infected_node = np.random.choice(infected_protected)
            #      neighbors = list(enw.neighbors(infected_node))
            #      susceptible_protected_neighbors = [node for node in neighbors if disease_states.get(node) == 'S' and node in protected]
            #      if len(susceptible_protected_neighbors)>0:
            #        susceptible_protected = np.random.choice(susceptible_protected_neighbors)
            #        disease_states[susceptible_protected] = "I"
            #        infected.append(susceptible_protected)
            #        N_infected += 1
            #        N_e_e += enw.degree(susceptible_protected)
            #        time_record.append(t)
            #        prevalence_count.append(N_infected)
            #        recovered_count.append(N_recovered)
            # Infection event #2: between S_N and I_P
            #elif draw_infection_event < (beta_PP + beta_NP)/sum_of_beta:
            #    infected_protected = list(set(infected).intersection(protected))
            #    if len(infected_protected)>0:
            #      infected_node = np.random.choice(infected_protected)
            #      neighbors = list(enw.neighbors(infected_node))
            #      susceptible_not_protected_neighbors = [node for node in neighbors if disease_states.get(node) == 'S' and node in not_protected]
            #      if len(susceptible_not_protected_neighbors)>0:
            #        susceptible_not_protected = np.random.choice(susceptible_not_protected_neighbors)
            #        disease_states[susceptible_not_protected] = "I"
            #        infected.append(susceptible_not_protected)
            #        N_infected += 1
            #        N_e_e += enw.degree(susceptible_not_protected)
            #        time_record.append(t)
            #        prevalence_count.append(N_infected)
            #        recovered_count.append(N_recovered)
            # Infection event #3: between S_P and I_N
            #elif draw_infection_event < (beta_PP + beta_NP + beta_PN)/sum_of_beta:
            #    infected_not_protected = list(set(infected).intersection(not_protected))
            #    if len(infected_not_protected)>0:
            #      infected_node = np.random.choice(infected_not_protected)
            #      neighbors = list(enw.neighbors(infected_node))
            #      susceptible_protected_neighbors = [node for node in neighbors if disease_states.get(node) == 'S' and node in protected]
            #      if len(susceptible_protected_neighbors)>0:
            #        susceptible_protected = np.random.choice(susceptible_protected_neighbors)
            #        disease_states[susceptible_protected] = "I"
            #        infected.append(susceptible_protected)
            #        N_infected += 1
            #        N_e_e += enw.degree(susceptible_protected)
            #        time_record.append(t)
            #        prevalence_count.append(N_infected)
            #        recovered_count.append(N_recovered)
            # Infection event #4: between S_N and I_N
            #else:
            #    infected_not_protected = list(set(infected).intersection(not_protected))
            #    if len(infected_not_protected)>0:
            #      infected_node = np.random.choice(infected_not_protected)
            #      neighbors = list(enw.neighbors(infected_node))
            #      susceptible_not_protected_neighbors = [node for node in neighbors if disease_states.get(node) == 'S' and node in not_protected]
            #      if len(susceptible_not_protected_neighbors)>0:
            #        susceptible_not_protected = np.random.choice(susceptible_not_protected_neighbors)
            #        disease_states[susceptible_not_protected] = "I"
            #        infected.append(susceptible_not_protected)
            #        N_infected += 1
            #        N_e_e += enw.degree(susceptible_not_protected)
            #        time_record.append(t)
            #        prevalence_count.append(N_infected)
            #        recovered_count.append(N_recovered)
            ############# Old Code ends here #################################################################

          # Case 5: Disease recovery
        elif event < p5:
                  recovered_node = np.random.choice(infected)
                  disease_states[recovered_node] = "R"
                  infected.remove(recovered_node)
                  recovered.append(recovered_node)
                  N_infected -= 1
                  N_recovered += 1
                  N_e_e -= enw.degree(recovered_node)
                  time_record.append(t)
                  prevalence_count.append(N_infected)
                  recovered_count.append(N_recovered)


          # Case 6: # Change to not adopting protection based on information layer
          # n_G is the total spreader neighbors on the information layer,
          # while k_info is the total neighbor count on the information layer
        elif event < p6:
            if len(protected) > 0:
              node_to_not_protect = np.random.choice(protected)
              n_G = sum(1 for node in filter(lambda x: info_states[x] == "G", inw.neighbors(node_to_not_protect)))
              k_info = len(list(inw.neighbors(node_to_not_protect)))
              frac_inf = N_infected/N
              #frac_inf=(N_recovered+N_infected)/N????
              if np.random.uniform() < (n_G / k_info) * np.tanh(1-frac_inf):#(1-np.tanh(frac_inf)) : #n_G / k_info:
                    lprot[node_to_not_protect] = "N"
                    protected.remove(node_to_not_protect)
                    not_protected.append(node_to_not_protect)
                    N_protected -= 1

          # Case 7: Change to adopting protection based on information layer
        elif event < p7:
            if len(not_protected) > 0:
                node_to_protect = np.random.choice(not_protected)
                n_G = sum(1 for node in filter(lambda x: info_states[x] == "G", inw.neighbors(node_to_protect)))
                k_info = len(list(inw.neighbors(node_to_protect)))
                frac_inf = N_infected/N
                if np.random.uniform() < (1 - n_G / k_info) * np.tanh(frac_inf): #1 - n_G / k_info:
                        lprot[node_to_protect] = "P"
                        not_protected.remove(node_to_protect)
                        protected.append(node_to_protect)
                        N_protected += 1

          # Case 8: # Change to not adopting protection based on neighborhood behavior in cognition layer
          # n_P is the total protected neighbors on the cognition layer,
          # while k_cog is the total neighbor count on the cognition layer
        elif event < p8:
        #else:
            if len(protected) > 0:
                node_to_not_protect = np.random.choice(protected)
                n_P = sum(1 for node in filter(lambda x: lprot[x] == "P", cnw.neighbors(node_to_not_protect)))
                k_cog = len(list(cnw.neighbors(node_to_not_protect)))
                if np.random.uniform() < 1 - n_P / k_cog:
                        lprot[node_to_not_protect] = "N"
                        protected.remove(node_to_not_protect)
                        not_protected.append(node_to_not_protect)
                        N_protected -= 1

          # Case 9: # Change to adopting protection based on neighborhood behavior in cognition layer
        else:
            if len(not_protected) > 0:
                node_to_protect = np.random.choice(not_protected)
                n_P = sum(1 for node in filter(lambda x: lprot[x] == "P", cnw.neighbors(node_to_protect)))
                k_cog = len(list(cnw.neighbors(node_to_protect)))
                if np.random.uniform() < n_P / k_cog:
                        lprot[node_to_protect] = "P"
                        not_protected.remove(node_to_protect)
                        protected.append(node_to_protect)
                        N_protected += 1

      #print("N_infected", N_infected, "N_gossip", N_gossip)
      if N_infected == 0:
        corrected_frac = N_corrected / N
        protected_frac = N_protected / N
        recovered_frac = N_recovered / N
        rho_C.append(corrected_frac)
        rho_P.append(protected_frac)
        rho_R.append(recovered_frac)
          #print("corrected_frac", corrected_frac, "recovered_frac", recovered_frac)

  avg_rho_C = sum(rho_C) / len(rho_C)
  avg_rho_P = sum(rho_P) / len(rho_P)
  avg_rho_R = sum(rho_R) / len(rho_R)

  return avg_rho_C, avg_rho_P, avg_rho_R

In [ ]:
lam = 0.5
alp = 0.1  # get bored of the information in 10 days, this is the stifling rate
zeta_1 = 0.2  # go to not protect by information
zeta_3 = 0.0 # go to not protect by neighborhood behavior
mu = 0.2
R0 = 2.0 # reproduction number
beta_NN = R0 * mu / 4.5
beta_PN = beta_NN * 0.8
beta_NP = beta_NN * 0.6
beta_PP = beta_NN * 0.4
n_sample = 100

# Set the SN-IP and SP-IN disease spreading rates
zeta_2_values6 = np.arange(0.0, 0.55, 0.05)
zeta_4_values6 = np.arange(0.0, 0.55, 0.05)

# Initialize the result array
results_rho_R = np.zeros((len(zeta_2_values6), len(zeta_4_values6)))

# Iterate over values
for i, zeta_2 in enumerate(zeta_2_values6):
    for j, zeta_4 in enumerate(zeta_4_values6):
        avg_rho_C, avg_rho_P, avg_rho_R = ICE_model_no_control(inw, ldeg_i, ltre, cnw, ldeg_c, lprot, enw, ldeg_e, lam, alp, zeta_1, zeta_2, zeta_3, zeta_4, beta_PP, beta_NP, beta_PN, beta_NN, mu, n_sample)
        results_rho_R[i, j] = avg_rho_R
# Save result
df3 = pd.DataFrame(results_rho_R)
df3.to_csv('/content/drive/My Drive/Information_Behavior_Disease_Networks/0418_results/Attack_rate_zeta2_zeta4_0429.csv')
df3.to_csv('/content/drive/My Drive/Network_Tests/Data/Attack_rate_zeta2_zeta4_0429.csv')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

df = pd.read_csv('/content/drive/My Drive/Information_Behavior_Disease_Networks/0418_results/Attack_rate_zeta2_zeta4_0429.csv')
results_rho_R = df.to_numpy()

zeta_2_values6 = np.arange(0.0, 0.55, 0.05)
zeta_4_values6 = np.arange(0.0, 0.55, 0.05)

# Create a heatmap
fig, ax = plt.subplots(figsize=(8, 6))

# Add smoothing by interpolating the image
im = ax.imshow(results_rho_R[:, 1:11], cmap='jet', aspect='auto', origin='lower',
               extent=[min(zeta_4_values6), max(zeta_4_values6), min(zeta_2_values6), max(zeta_2_values6)],
               interpolation='bilinear')

# Setting labels, colorbar
ax.set_xlabel('Protection adoption by neighborhood behavior', fontsize=14)
ax.set_ylabel('Protection adoption by information', fontsize=14)
plt.colorbar(im, label='Attack Rate', ax=ax)
plt.rc('xtick', labelsize=14)
plt.rc('ytick', labelsize=14)

# Display the plot
plt.savefig('/content/drive/My Drive/Information_Behavior_Disease_Networks/0418_results/heatmap_zeta2_zeta4_0429.pdf')
plt.savefig('/content/drive/My Drive/Network_Tests/Data/heatmap_zeta2_zeta4_0429.pdf')
#plt.show()